In [50]:
import pandas as pd
import os
file_name = "Africa_grid_-5_32_6_42"
image_file_name = file_name
# file_name = "Shanghai_time_series" 
# image_file_name = "Shanghai_grid_120_30_122.25_32"
df = pd.read_csv(f"/nas/houce/Alphaearth_embedding/GEE_extracted/{file_name}/metadata/{image_file_name}_grid_cells.csv")

In [51]:
from glob import glob

finished_files = [x.split('/')[-1] for x in glob(f"/nas/houce/Alphaearth_embedding/GEE_extracted/{image_file_name}/*")]
download_zips = [x.split('/')[-1][:-4] for x in glob("/nas/houce/Alphaearth_embedding/zips/*")]
common_files = set(finished_files) & set(download_zips)
del_files = [f"/nas/houce/Alphaearth_embedding/zips/{x}.zip" for x in common_files]

for file_path in del_files:
    if os.path.exists(file_path):
        os.remove(file_path)
        print(f"已删除: {file_path}")
    else:
        print(f"文件不存在: {file_path}")

已删除: /nas/houce/Alphaearth_embedding/zips/drive-download-20250902T231541Z-1-004.zip
已删除: /nas/houce/Alphaearth_embedding/zips/drive-download-20250902T231541Z-1-007.zip
已删除: /nas/houce/Alphaearth_embedding/zips/drive-download-20250902T143642Z-1-003.zip
已删除: /nas/houce/Alphaearth_embedding/zips/drive-download-20250902T161116Z-1-011.zip
已删除: /nas/houce/Alphaearth_embedding/zips/drive-download-20250902T161116Z-1-009.zip
已删除: /nas/houce/Alphaearth_embedding/zips/drive-download-20250902T161116Z-1-002.zip
已删除: /nas/houce/Alphaearth_embedding/zips/drive-download-20250902T161116Z-1-008.zip
已删除: /nas/houce/Alphaearth_embedding/zips/drive-download-20250902T161116Z-1-010.zip
已删除: /nas/houce/Alphaearth_embedding/zips/drive-download-20250902T231541Z-1-001.zip
已删除: /nas/houce/Alphaearth_embedding/zips/drive-download-20250902T143642Z-1-001.zip
已删除: /nas/houce/Alphaearth_embedding/zips/drive-download-20250902T161116Z-1-006.zip
已删除: /nas/houce/Alphaearth_embedding/zips/drive-download-20250902T231541Z-1-

In [52]:
# image_file_name = "Africa_grid_-20_10_-15_15"
os.makedirs(f"/nas/houce/Alphaearth_embedding/GEE_extracted/{image_file_name}/metadata", exist_ok=True)

extracted_files = glob(f"/nas/houce/Alphaearth_embedding/GEE_extracted/{image_file_name}/*.tif")
extracted_files_2 = glob(f"/nas/houce/Alphaearth_embedding/GEE_extracted/{image_file_name}/*/*.tif")
extracted_files_df = pd.DataFrame(extracted_files, columns=['file_path'])
extracted_files_df2 = pd.DataFrame(extracted_files_2, columns=['file_path'])
extracted_files_df = pd.concat([extracted_files_df, extracted_files_df2], ignore_index=True)

In [53]:
extracted_files_df['grid_name'] = extracted_files_df['file_path'].apply(lambda x: f"{x.split('/')[-1][20:]}")
extracted_files_df['grid_name'] = extracted_files_df['grid_name'].apply(lambda x: '-'.join(x.split('-')[:-4])[:-5] if '-' in x else x)
extracted_files_df['start_time'] = extracted_files_df['file_path'].apply(lambda x: f"{x.split('/')[-1].split('_')[-2]}")
extracted_files_df['end_time'] = extracted_files_df['file_path'].apply(lambda x: f"{x.split('/')[-1].split('_')[-1][:-4]}")

In [54]:
df.merge(extracted_files_df, on='grid_name', how='left').to_csv(f"/nas/houce/Alphaearth_embedding/GEE_extracted/{image_file_name}/metadata/{image_file_name}_grid_cells_check.csv", index=False)
tem = df.merge(extracted_files_df, on='grid_name', how='left')
tem[tem['file_path'].notna()].to_csv(f"tem.csv")
tem[tem['file_path'].isna()].to_csv(f"tem_na.csv")
tem[tem['file_path'].notna()].to_csv(f"/nas/houce/Alphaearth_embedding/GEE_extracted/{image_file_name}/metadata/tem.csv",index=None)
tem[tem['file_path'].isna()].to_csv(f"/nas/houce/Alphaearth_embedding/GEE_extracted/{image_file_name}/metadata/tem_na.csv",index=None)

In [55]:
duplicates = tem[tem['file_path'].notna()].duplicated(subset=['grid_name'], keep=False)
dup_df = tem[tem['file_path'].notna()][duplicates]
dup_df.drop_duplicates(subset=['grid_name'], inplace=True)
dup_dir= dup_df['file_path'].drop_duplicates().tolist()

In [56]:
import os
import shutil

for dir_path in dup_dir:
    if os.path.exists(dir_path) and os.path.isdir(dir_path):
        shutil.rmtree(dir_path)
        print(f"已删除目录及其内容: {dir_path}")
    elif os.path.exists(dir_path):
        os.remove(dir_path)
        print(f"已删除文件: {dir_path}")
    else:
        print(f"路径不存在: {dir_path}")